In [3]:
import itemQuery as IQ
%run nfl-parse-demo.py

Name, Major:
0, http://www.steelers.com/cda-web/rss-module.htm?tagName=News
1, http://www.steelers.com/cda-web/rss-module.htm?tagName=Photos
2, http://www.steelers.com/cda-web/rss-module.htm?tagName=Videos
3, http://www.steelers.com/cda-web/rss-module.htm?tagName=Audio
4, http://www.steelers.com/cda-web/rss-module.htm?tagName=Espa%C3%B1ol
5, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News
6, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Press%20Releases
7, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Transcripts
8, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Late%20For%20Work
9, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20John%20Eisenberg
10, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20The%20Caw
11, http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Byrne%20Identity
12, http://www.baltimoreravens.com/cda-w

In [4]:
IQ.runItems

<function itemQuery.runItems>

In [5]:
vals


[['0',
  'Steelers',
  'News',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=News',
  'Steelers.com'],
 ['1',
  'Steelers',
  'Photos',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Photos',
  'Steelers.com'],
 ['2',
  'Steelers',
  'Videos',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Videos',
  'Steelers.com'],
 ['3',
  'Steelers',
  'Audio',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Audio',
  'Steelers.com'],
 ['4',
  'Steelers',
  'Espanol',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Espa%C3%B1ol',
  'Steelers.com'],
 ['5',
  'Ravens',
  'All News',
  'http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News'],
 ['6',
  'Ravens',
  'Press Releases',
  'http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Press%20Releases'],
 ['7',
  'Ravens',
  'Transcripts',
  'http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Transcripts'],
 ['8',
  'Ravens',
  'Late For Work'

In [109]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Encountered a start tag:", tag)
        for attr in attrs:
            print(attr, end= ' ')
        print(' ')

    #def handle_endtag(self, tag):
        #print("Encountered an end tag :", tag)

    #def handle_data(self, data):
        #print("Encountered some data  :", data)

In [107]:
class twitterWidgetParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Encountered a start tag:", tag)


    def handle_endtag(self, tag):
        print("Encountered an end tag :", tag)

    def handle_data(self, data):
        print("Encountered some data  :", data)

In [63]:
class articleContentParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.articleContent = ''
        self.articleContentDiv = False
        self.script = False
        self.twitterContent = False
        self.divs = 0
    
    def handle_starttag(self, tag, attrs):        
        if tag == 'div':
            if ('class', 'article-content') in attrs:
                self.articleContentDiv = True
                print("ContentOn")
        
        if self.articleContentDiv and tag == 'div':
            self.divs += 1
        
        if tag == 'script':
            self.script = True
            #print("scriptOn")
            
        if tag == 'blockquote':                  
            self.twitterContent = True
            #print("TwitterOn")

    def handle_endtag(self, tag):
        if self.articleContentDiv and tag == 'div':
            self.divs -= 1
            if self.divs == 0:
                self.articleContentDiv = False
                print("ContentOff")
                
        if tag == 'script':
            self.script = False
            #print("scriptOFF")
            
        if tag == 'blockquote':
            self.twitterContent = False
            #print("TwitterOFF")

    def handle_data(self, data):
        #print("Encountered some data  :", data)
        if self.articleContentDiv and not (self.script or self.twitterContent):
            self.articleContent += data
            self.articleContent += ' '
    
    def getContent(self):
        return self.articleContent

In [101]:
class paneContentParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.articleContent = ''
        self.articleContentDiv = False
        self.script = False
        self.twitterContent = False
        self.divs = 0
    
    def handle_starttag(self, tag, attrs):        
        if tag == 'div':
            if ('class', 'pane-content') in attrs:
                self.articleContentDiv = True
                print("PaneOn")
        
        if self.articleContentDiv and tag == 'div':
            self.divs += 1
        
        if tag == 'script':
            self.script = True
            print("scriptOn")
            
        if tag == 'blockquote':                  
            self.twitterContent = True
            self.articleContent += "__BLOCKQUOTE__"
            print("TwitterOn")

    def handle_endtag(self, tag):
        if self.articleContentDiv and tag == 'div':
            self.divs -= 1
            if self.divs == 0:
                self.articleContentDiv = False
                print("PaneOff")
                
        if tag == 'script':
            self.script = False
            print("scriptOFF")
            
        if tag == 'blockquote':
            self.twitterContent = False
            print("TwitterOFF")

    def handle_data(self, data):
        #print("Encountered some data  :", data)
        if self.articleContentDiv and not (self.script or self.twitterContent):
            self.articleContent += data
            self.articleContent += ' '
    
    def getContent(self):
        return self.articleContent

In [84]:
seaArticleDemo = IQ.getArticle("http://www.seahawks.com/news/2017/08/14/seahawks-react-preseason-victory-over-los-angeles-chargers")

In [85]:
len(seaArticleDemo)

172197

In [86]:
seaArticleContent = IQ.findContent(seaArticleDemo, '<div class="pane-content">')

In [87]:
len(seaArticleContent)

3784

In [8]:
patsArticleDemo = IQ.getArticle("http://www.patriots.com/news/2017/08/15/game-preview-patriots-texans")

In [9]:
len(patsArticleDemo)

153206

In [11]:
patsArticleContent = IQ.findContent(patsArticleDemo, '<div class="pane-content">')

In [67]:
len(patsArticleContent)

9162

In [14]:
articleDemo = IQ.getArticle("http://www.baltimoreravens.com/news/article-1/Late-For-Work-814-Ravens-Cap-Room-Doubled-Trade-Talk-Swirling-Whats-Ozzie-Newsome-up-to-/4181819c-a79f-498c-b067-7b9d457c9e16")

In [15]:
len(articleDemo)

126228

In [16]:
articleContent = IQ.findContent(articleDemo, '<div class="article-content">')

In [17]:
len(articleContent)

24844

In [102]:
parser = paneContentParser()
parser.feed(patsArticleDemo)

scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
PaneOn
scriptOn
scriptOFF
PaneOff
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF


In [103]:
parser.getContent()

"\n    \n   \n     NEW ENGLAND PATRIOTS (0-1) at HOUSTON TEXANS (0-1) Saturday, Aug. 19, 2017 at 8:00 p.m. ET. NRG Stadium (71,795) PATRIOTS:   Download Release  |  Roster  |  Depth Chart   |  Watch Games TEXANS:  Opponent Release  |  Roster  |  Depth Chart New England and Houston will square off in a preseason game for the first time since the Texans' inaugural season in 2002. Houston is the only team in the NFL that the Patriots have never played in a preseason game. The Patriots will return to Houston for the first time since their 34-28 overtime victory over the Atlanta Falcons in Super Bowl LI on Feb. 5, 2017. New England and Houston met twice last year and are scheduled to play each other twice again this year. New England scored a decisive 27-0 victory over Houston in a Week 3 Thursday Night matchup at Gillette Stadium on Sept. 27, 2017, and a 34-16 Divisional Playoff win over the Texans at Gillette on Jan. 14, 2017. In addition to their game on Saturday, the Patriots and Texans

In [104]:
parser = paneContentParser()
parser.feed(seaArticleDemo)

scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
PaneOn
scriptOn
scriptOFF
PaneOff
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF
scriptOn
scriptOFF


In [105]:
parser.getContent()

'\n    \n   \n     \n    \n\n   \n     \n   \n \n   \n \n In the first preseason game of the 2017 season, four rushing touchdowns and four forced turnovers set the tone in a convincing 48-17 Seahawks victory over the Los Angeles Chargers at the StubHub Center Below, check out how the Seahawks reacted to the win on social media: https://twitter.com/dshead24/status/896933667045384192 https://twitter.com/Oday_Aboushi75/status/896945838160625664 https://twitter.com/DangeRussWilson/status/896947705703825408 https://twitter.com/JustinBritt68/status/896948042049372161 https://twitter.com/GermainX1/status/896950089716912129 https://twitter.com/PeteCarroll/status/896954084246175744 https://twitter.com/Dewey_MAC/status/896957418495549440 https://twitter.com/thec1_/status/896958728238284801 https://twitter.com/tswoopes18/status/896959854840922112 https://twitter.com/prichiejr/status/897014197069819904 https://www.instagram.com/p/BXyUpVFDHGA/ https://www.instagram.com/p/BXyP-zOAWlp/ https://www.in

In [110]:
parser = MyHTMLParser()
parser.feed(seaArticleDemo)

Encountered a start tag: html
('xmlns:nflcs', 'http://clubs.nfl.com') ('class', 'no-js') ('lang', 'en') ('dir', 'ltr') ('prefix', 'og: http://ogp.me/ns# fb: http://www.facebook.com/2008/fbml c: http://s.opencalais.com/1/pred/ sys: http://s.opencalais.com/1/type/sys/ lid: http://s.opencalais.com/1/type/lid/ cat: http://s.opencalais.com/1/type/cat/ resolved: http://s.opencalais.com/1/type/er/ cgeo: http://s.opencalais.com/1/type/er/Geo/ eventfact: http://s.opencalais.com/1/type/em/r/ entity: http://s.opencalais.com/1/type/em/e/ cld: http://s.opencalais.com/1/linkeddata/pred/ content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/

('href', 'https://www.facebook.com/Seahawks') ('target', '_blank')  
Encountered a start tag: li
('class', 'leaf menu-mlid-13591')  
Encountered a start tag: a
('href', 'https://plus.google.com/+Seahawks/posts') ('target', '_blank')  
Encountered a start tag: li
('class', 'leaf menu-mlid-13586')  
Encountered a start tag: a
('href', 'http://instagram.com/seahawks') ('target', '_blank')  
Encountered a start tag: li
('class', 'leaf menu-mlid-13596')  
Encountered a start tag: a
('href', 'https://www.pinterest.com/seattleseahawks/') ('target', '_blank')  
Encountered a start tag: li
('class', 'collapsed menu-mlid-13576')  
Encountered a start tag: a
('href', 'https://twitter.com/seahawks') ('target', '_blank')  
Encountered a start tag: li
('class', 'leaf menu-mlid-17261')  
Encountered a start tag: a
('href', 'http://www.seahawks.com/content/rss-feeds')  
Encountered a start tag: li
('class', 'last leaf menu-mlid-17981')  
Encountered a start tag: a
('href', '/about/email-subscriptions'

('href', '/community/youth/play-60/tuesdays')  
Encountered a start tag: li
('class', 'expanded menu-mlid-11676')  
Encountered a start tag: a
('href', '/community/donations')  
Encountered a start tag: ul
('class', 'menu')  
Encountered a start tag: li
('class', 'first leaf menu-mlid-14116')  
Encountered a start tag: a
('href', 'http://www.nflfoundation.org/applications/programs/view/grassroots') ('target', '_blank')  
Encountered a start tag: li
('class', 'leaf menu-mlid-14101')  
Encountered a start tag: a
('href', '/community/foundation')  
Encountered a start tag: li
('class', 'leaf menu-mlid-11656')  
Encountered a start tag: a
('href', '/community/salute')  
Encountered a start tag: li
('class', 'leaf menu-mlid-11631')  
Encountered a start tag: a
('href', '/community/donations')  
Encountered a start tag: li
('class', 'leaf menu-mlid-11646')  
Encountered a start tag: a
('href', '/community/coaches-players-foundations')  
Encountered a start tag: li
('class', 'last leaf menu-m

Encountered a start tag: a
('href', 'https://twitter.com/prichiejr/status/897014197069819904')  
Encountered a start tag: p
 
Encountered a start tag: a
('href', 'https://www.instagram.com/p/BXyUpVFDHGA/')  
Encountered a start tag: p
 
Encountered a start tag: a
('href', 'https://www.instagram.com/p/BXyP-zOAWlp/')  
Encountered a start tag: p
 
Encountered a start tag: a
('href', 'https://www.instagram.com/p/BXyPFgMD2NB/')  
Encountered a start tag: p
 
Encountered a start tag: a
('href', 'https://www.instagram.com/p/BXyGNgGlu0u/')  
Encountered a start tag: a
('href', '/news/2017/08/14/seahawks-react-preseason-victory-over-los-angeles-chargers') ('class', 'read-more-link element-invisible body-190521') ('data', '190521')  
Encountered a start tag: div
('class', 'panel-pane pane-views-panes pane-latest-content-list-widget-latest-content-list-panel-pane component-general ft-4') ('class', 'panel-pane pane-views-panes pane-latest-content-list-widget-latest-content-list-panel-pane compone

('href', 'http://www.miamidolphins.com/') ('target', '_blank')  
Encountered a start tag: li
 
Encountered a start tag: a
('href', 'http://www.patriots.com/') ('target', '_blank')  
Encountered a start tag: li
 
Encountered a start tag: a
('href', 'http://www.newyorkjets.com/') ('target', '_blank')  
Encountered a start tag: ul
('class', 'menu')  
Encountered a start tag: li
('class', 'location')  
Encountered a start tag: li
 
Encountered a start tag: a
('href', 'http://www.baltimoreravens.com/') ('target', '_blank')  
Encountered a start tag: li
 
Encountered a start tag: a
('href', 'http://www.bengals.com/') ('target', '_blank')  
Encountered a start tag: li
 
Encountered a start tag: a
('href', 'http://www.clevelandbrowns.com/') ('target', '_blank')  
Encountered a start tag: li
 
Encountered a start tag: a
('href', 'http://www.steelers.com/') ('target', '_blank')  
Encountered a start tag: ul
('class', 'menu')  
Encountered a start tag: li
('class', 'location')  
Encountered a sta

In [111]:
seaArticleDemo

'<!DOCTYPE html>\n  <!--[if IEMobile 7]><html xmlns:nflcs="http://clubs.nfl.com" class="no-js ie iem7" lang="en" dir="ltr" prefix="og: http://ogp.me/ns# fb: http://www.facebook.com/2008/fbml c: http://s.opencalais.com/1/pred/ sys: http://s.opencalais.com/1/type/sys/ lid: http://s.opencalais.com/1/type/lid/ cat: http://s.opencalais.com/1/type/cat/ resolved: http://s.opencalais.com/1/type/er/ cgeo: http://s.opencalais.com/1/type/er/Geo/ eventfact: http://s.opencalais.com/1/type/em/r/ entity: http://s.opencalais.com/1/type/em/e/ cld: http://s.opencalais.com/1/linkeddata/pred/ content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/